In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from model import RiceClassifier
from data import RiceDataset, get_transform
import torch
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm

In [ ]:
def load_val_dataset(input_dir=Path('../../data/Rice_Image_Dataset/'), val_ratio=0.2, num_workers=6, batch_size=32):
    dataset = RiceDataset(input_dir, transform=get_transform())
    val_amount = int(val_ratio * len(dataset))
    train_amount = len(dataset) - val_amount
    _, val_dataset = torch.utils.data.random_split(dataset, (train_amount, val_amount),
                                                               generator=torch.Generator().manual_seed(42))
    
    val_loader = torch.utils.data.DataLoader(val_dataset, num_workers=num_workers, batch_size=batch_size,
                                             drop_last=False)
    return val_loader

In [ ]:
dataset = load_val_dataset()

In [ ]:
RESNET_WEIGHTS_PATH = 'resnet.pt'

In [ ]:
model = RiceClassifier('resnet18')
model.load_state_dict(torch.load(RESNET_WEIGHTS_PATH))

In [ ]:
model.eval();

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model.to(device);

In [ ]:
preds_list = []
targets_list = []
for imgs, targets in tqdm(dataset):
    with torch.no_grad():
        imgs = imgs.to(device)
        targets = targets.to(device)
        preds = model.forward(imgs)
        preds = torch.argmax(preds, -1)
        preds = preds.cpu().numpy()
        targets = targets.cpu().numpy()
        preds_list.append(preds)
        targets_list.append(targets)

In [ ]:
y_preds = np.concatenate(preds_list)
y_true = np.concatenate(targets_list)

In [ ]:
y_preds, y_true

In [ ]:
print(classification_report(y_true, y_preds))

In [ ]:
EFFICIENT_WEIGHTS_PATH = 'efficientnet.pt'

In [ ]:
model = RiceClassifier('efficientnet_b0')
model.load_state_dict(torch.load(EFFICIENT_WEIGHTS_PATH))

In [ ]:
# model.eval();

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model.to(device);

In [ ]:
preds_list = []
targets_list = []
for imgs, targets in tqdm(dataset):
    with torch.no_grad():
        imgs = imgs.to(device)
        targets = targets.to(device)
        preds = model.forward(imgs)
        preds = torch.argmax(preds, -1)
        preds = preds.cpu().numpy()
        targets = targets.cpu().numpy()
        preds_list.append(preds)
        targets_list.append(targets)

In [ ]:
y_preds = np.concatenate(preds_list)
y_true = np.concatenate(targets_list)

In [ ]:
y_preds, y_true

In [ ]:
print(classification_report(y_true, y_preds))

In [ ]:
cm = confusion_matrix(y_true, y_preds)
disp = ConfusionMatrixDisplay(cm)
disp.plot()